In [1]:
import pyscf
import pyscf.tools
import scipy
import numpy as np

from orbitalpartitioning import *

In [2]:
molecule = """
 Fe 1.67785607 0.00052233 0.06475932
 O 0.00000000 0.00000000 -0.47099074
 Fe -1.67785607 -0.00052233 0.06475932
 Cl 1.87002704 -1.09796437 1.99091682
 Cl 2.93244917 -0.98210488 -1.47467288
 Cl 2.37160936 2.07954091 -0.50446591
 Cl -1.87002704 1.09796437 1.99091682
 Cl -2.93244917 0.98210488 -1.47467288
 Cl -2.37160936 -2.07954091 -0.50446591
"""

basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   10, # number of unpaired electrons
        charge  =   -2,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol).newton()
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()

# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing

# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]

symmetry:  C2


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> Newton solver flags ********
SCF tol = 1e-08
conv_tol_grad = 1e-05
max. SCF cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_cycle_inner = 12
max_stepsize = 0.05
ah_start_tol = 1e+09
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_start_cycle = 1
ah_max_cycle = 40
ah_grad_trust_region = 2.5
kf_interval = 4
kf_trust_region = 5
canonicalization = True
max_memory 4000 MB (current use 120 MB)
HOMO (B) = 0.318236980196575  LUMO (A) = 0.320979897364209
Initial guess E= -5347.0130535218  |g|= 4.92607
macro= 0  E= -5354.09525943034  delta_E= -7.08221  |g|= 1.2839  3 KF 15 JK
macro= 1  E= -5354.99700870194  delta_E= -0.901749  |g|= 0.380059  3 KF 16 JK
macro= 2  E= -5355.40185091213  delta_E= -0.404842  |g|= 0.324421  3 KF 17 JK
macro= 3  E= -5355.82121744052  delta_E= -0.419367  |g|= 0.146705  3 KF 18 JK
macro= 4  E= -5355.8871022433  delta_E= -0.0658848  |g|= 0.199741

# Define Fragments by AOs

In [3]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d", "4s","4d"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("2s", "2p","3p"):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 2:
        if ao[2] in ("3d", "4s","4d"):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)


[[3, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [32, 34, 35, 36, 37, 38, 39], [48, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68]]


# Define Projectors
We can choose to project onto the non-orthogonal AOs, or onto the symmetrically orthogonalized AOs.

In [4]:
# Define projectors
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


# Project MOs onto all fragments
For each orbital block (Docc, Sing, Virt), project each subspace onto the full list of fragment AOs. This will determine our full active space.

In [5]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))

 Partition  184 orbitals into a total of   29 orbitals
            Index   Sing. Val. Space       
                0   1.60416681            2*
                1   1.57587273            2*
                2   1.57106902            2*
                3   1.35803010            2*
                4   1.33368722            2*
                5   1.20985256            2*
                6   1.20584819            2*
                7   1.19055703            2*
                8   1.18865706            2*
                9   1.16430588            2*
               10   1.16291465            2*
               11   1.14539476            0*
               12   1.12593359            2*
               13   1.12432916            2*
               14   0.95943767            1*
               15   0.93838983            1*
               16   0.93412686            1*
               17   0.92683096            1*
               18   0.91436519            1*
               19   0.91429348            1*
 

# Split active space into fragments

In [6]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1

# import pickle

# data = {}
# data["Pf"] = Pf 
# data["Cdocc"] = Cdocc
# data["Csing"] = Csing
# data["Cvirt"] = Cvirt
# data["S"] = S
# data["frags"] = frags 

for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof



# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

Cact = np.hstack(Cfrags)

# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


# data["Cact"] = Cact
# data["init_fspace"] = init_fspace 
# data["clusters"] = clusters 
# with open('data_CrOCr.pickle', 'wb') as handle:
#     pickle.dump(data, handle)



 Fragment:  [3, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
 Partition   29 orbitals into a total of   11 orbitals
            Index   Sing. Val. Space       
                0   1.18601712            2*
                1   1.18016712            2*
                2   1.13749962            2*
                3   1.11196672            2*
                4   1.04694357            2*
                5   0.91623872            1*
                6   0.91383430            1*
                7   0.91252842            1*
                8   0.91132536            1*
                9   0.90577777            1*
               10   0.83248618            0*
               11   0.44689874            0
               12   0.27818251            0
               13   0.26049117            0
               14   0.11787773            0
               15   0.07033956            1
               16   0.06302698            1
               17   0.05152314            2
               18   0.04645116           

# Make Integrals

In [7]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)

(184, 71)
(184, 29)
(184, 184)


In [8]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;

In [9]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [10]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [11]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

In [12]:
import numpy as np
Ccmf = np.load("Ccmf_29.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf_29.molden", Ccmf)



WARN: orbitals [ 0  1  2  3  4  5  6  7  8  9 10 18 19 20 21 22 23 24 25 26 27 28] not symmetrized, norm = [0.50000001 0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.5        0.5        0.50000001
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.5       ]

